# Fase Pre-Release de la entrega del reto
### ***Secuencia***
1. Generar frecuencias de ventas y simular una venta de 105 clientes.
2. Tomar los volúmenes de las compras de los clientes para determinar la cantidad de camiones necesarios.
3. Sabiendo los camiones, utilizar k-medoids para hacer los mini-tcps.
4. Resolver los mtcps.

### ***Librerías***

In [55]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import math

from scipy.stats import poisson
from scipy.stats import poisson
from scipy.stats import chisquare


from ortools.linear_solver import pywraplp

from scipy.spatial.distance import cdist

import time

## Paso 1. Simular un pedido de 100 clientes

In [56]:
compras = pd.read_excel('informacion_compra.xlsx')

In [57]:
compras.head(3)

,Producto,Unidades,Factura
0,48443,1,799186
1,42877,1,717106
2,48296,1,468125


In [58]:
facturas = []
productos = []
for _,i in compras.iterrows():
    facturas += [i[2]]*i[1]
for _,i in compras.iterrows():
    productos += [i[0]]*i[1]

compras = pd.DataFrame({
    "Factura": facturas,
    "Producto": productos
})

In [59]:
df = compras.groupby(['Factura']).size().reset_index(name='Frequency').Frequency.value_counts()
df = df.reset_index()
df.columns = ['Pedidos', 'Frecuencia']
df['Frecuencia ln'] = np.log(df['Frecuencia'])
valor_medio = np.sum(df['Pedidos'] * df['Frecuencia ln']) / np.sum(df['Frecuencia ln'])
df['Poisson'] = poisson.pmf(df['Pedidos'], valor_medio)
df['valores_esperados'] = df['Poisson'] * np.sum(df['Frecuencia ln'])
df.head(3)

,Pedidos,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,19235,9.864487,0.011672,0.783311
1,2,3546,8.173575,0.036706,2.463277
2,3,1024,6.931472,0.076953,5.164178


In [60]:
df2 = compras.groupby(['Producto']).size().reset_index(name='Frequency').Frequency.value_counts()
df2 = df2.reset_index()
df2.columns = ['Producto', 'Frecuencia']
df2['Frecuencia ln'] = np.log(df2['Frecuencia'])
valor_medio = np.sum(df2['Producto'] * df2['Frecuencia ln']) / np.sum(df2['Frecuencia ln'])
df2['Poisson'] = poisson.pmf(df2['Producto'], valor_medio)
df2['valores_esperados'] = df2['Poisson'] * np.sum(df2['Frecuencia ln'])
df2.head(3)

,Producto,Frecuencia,Frecuencia ln,Poisson,valores_esperados
0,1,1793,7.491645,1.565792e-11,2.886239e-09
1,2,779,6.658011,2.209339e-10,4.072494e-08
2,3,414,6.025866,2.078257e-09,3.830870e-07


In [61]:
def montecarlo(df, n):  
    resultados = []
    for _ in range(n):
        num = np.random.rand()
        for j in range(len(df)):
            if df[j][3] <= num < df[j][4]:
                resultados.append(j)
            
    return resultados

In [62]:
pedidos = df[['Pedidos','Poisson']]
pedidos['Acumulado'] = pedidos.Poisson.cumsum()
pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]
pedidos['Superior'] = pedidos['Acumulado']
pedidos = pedidos.to_numpy()

C:\Users\gerar\AppData\Local\Temp\ipykernel_21848\3932492861.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Acumulado'] = pedidos.Poisson.cumsum()
C:\Users\gerar\AppData\Local\Temp\ipykernel_21848\3932492861.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedidos['Inferior'] = [0] + pedidos['Acumulado'].tolist()[:-1]


In [63]:
#num_pedidos = montecarlo(pedidos,df.Frecuencia.sum())
num_pedidos = montecarlo(pedidos,compras.size)


In [64]:
producto = df2[['Producto','Poisson']]
producto['Acumulado'] = producto.Poisson.cumsum()
producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]
producto['Superior'] = producto['Acumulado']
producto = producto.to_numpy()

C:\Users\gerar\AppData\Local\Temp\ipykernel_21848\3891866760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Acumulado'] = producto.Poisson.cumsum()
C:\Users\gerar\AppData\Local\Temp\ipykernel_21848\3891866760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  producto['Inferior'] = [0] + producto['Acumulado'].tolist()[:-1]


In [65]:
envios = [np.array(montecarlo(producto, num)) for num in montecarlo(pedidos,110) if num!=0]
envios_np = envios[:106]

In [66]:
enviosArreglo = np.array([[-1,-1,-1]])
for i in range(len(envios)):
    a = np.unique(np.array(envios[i]), return_counts=True)
    enviosArreglo = np.concatenate((enviosArreglo, np.array([np.array([i]*len(a[0])),a[0],a[1]]).T))
enviosArreglo

array([[ -1,  -1,  -1],
       [  0,  18,   1],
       [  0,  25,   1],
       ...,
       [109,  26,   1],
       [109,  32,   1],
       [109,  50,   1]], dtype=int64)

In [67]:
dp = pd.read_csv('info_productos.csv')
dp.columns = ['Producto', 'Volumen']
nuevo_registro = {"Producto": 0, "Volumen": 0}
dp = pd.concat([pd.DataFrame([nuevo_registro]), dp], ignore_index=True)
volumenes = dp.to_numpy()[:,1]

In [68]:
dimensiones = []
for pedido in envios_np:
    acumulado = 0
    for i in pedido:
        acumulado += volumenes[i]
    dimensiones.append(round(acumulado,4))

## Paso 3. Sabiendo los camiones necesarios, hacer clusters

In [70]:
path = "https://raw.githubusercontent.com/gerardoramc/AlgoritmoDatos/Gerardo/distancias_tiempos2.csv"
df = pd.read_csv(path)
df.head(3)

,Distance,Duration
0,0.00,0:00
1,15.26,0:22
2,14.93,0:17


In [71]:
df['Duration'] = pd.to_datetime(df['Duration'])
df['Duration'] = df['Duration'].dt.strftime('%H:%M')
df['Duration']=df['Duration'].astype('string')

In [72]:

def convert_to_seconds(time_str):
    hours, minutes = map(int, time_str.split(':'))
    total_seconds = hours * 3600 + minutes * 60
    return total_seconds

# Apply the conversion function to the DataFrame column
df['Segundos'] = df['Duration'].apply(convert_to_seconds)

In [73]:
rangos = np.arange(0,11236,106)

In [74]:
cuadrada = []
for i in range(len(rangos)-1):
    cuadrada.append(list((df['Segundos'][rangos[i]:rangos[i+1]])))
result_array = np.append(np.array(cuadrada), [(df['Segundos'][rangos[-1]:])], axis = 0)
result_array.shape

(106, 106)

In [75]:
def remove_first_row_and_column(input_array):
    if input_array.shape[0] <= 1 or input_array.shape[1] <= 1:
        raise ValueError("Input array must have at least 2 rows and 2 columns.")

    new_array = input_array[1:, 1:]
    return new_array


In [77]:
vol_por_clus = {}

for clave, indices in clusters.items():
    pesos = [dimensiones[indice] for indice in indices]
    vol_por_clus[clave] = pesos


In [78]:
multiplier = 1
trucks_used = 0
start_time=time.time()

for clave, volumenes in vol_por_clus.items():

    print(f"**********CLUSTER # {clave + 1 }**********\n")
    incomplete = True
    while incomplete:
        data = {}
        data["weights"] = volumenes
        #data["values"] = len(data["weights"]) * [1]
        data["values"] =  volumenes
        sum_of_all_weights = sum(data["weights"])

        assert len(data["weights"]) == len(data["values"])
        data["num_items"] = len(data["weights"])
        data["all_items"] = range(data["num_items"])

        data["bin_capacities"] = [27] * multiplier
        data["num_bins"] = len(data["bin_capacities"])
        data["all_bins"] = range(data["num_bins"])

        solver = pywraplp.Solver.CreateSolver("SCIP")

        # x[i, b] = 1 if item i is packed in bin b.
        x = {}
        for i in data["all_items"]:
            for b in data["all_bins"]:
                x[i, b] = solver.BoolVar(f"x_{i}_{b}")

        # Each item is assigned to at most one bin.
        for i in data["all_items"]:
            solver.Add(sum(x[i, b] for b in data["all_bins"]) <= 1)

        # The amount packed in each bin cannot exceed its capacity.
        for b in data["all_bins"]:
            solver.Add(
                sum(x[i, b] * data["weights"][i] for i in data["all_items"])
                <= data["bin_capacities"][b])
        
        # Each bin must contain at least 2 items
        for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) <= 13)
            
         # Each bin must contain at most 20 items
        '''for b in data["all_bins"]:
            solver.Add(sum(x[i, b] for i in data["all_items"]) <= 20)'''
        
        # Maximize total value of packed items.
        objective = solver.Objective()
        for i in data["all_items"]:
            for b in data["all_bins"]:
                objective.SetCoefficient(x[i, b], data["values"][i])
        objective.SetMaximization()





        start_time=time.time()
        status = solver.Solve()

        if status == pywraplp.Solver.OPTIMAL:

            unused_items = [i for i in data["all_items"] if all(x[i, b].solution_value() == 0 for b in data["all_bins"])]


            used_bins = [b for b in data["all_bins"] if any(x[i, b].solution_value() > 0 for i in data["all_items"])]
            if len(unused_items) != 0:
                multiplier += 1

            if len(unused_items) == 0:


                print(f"Número de camiones requeridos: {len(used_bins)}\n")

                print(f"Valor total empaquetado: {round(objective.Value(), 2)}\n")

                max_value_bin = None
                max_value = 0
                total_weight = 0
                total_value = 0
                for b in used_bins:
                    print(f"*** CAMIÓN # {b+1} ***")
                    print(f'Capacidad: {data["bin_capacities"][b]}')
                    bin_weight = 0
                    bin_value = 0
                    packed_items = []
                    for i in data["all_items"]:
                        if x[i, b].solution_value() > 0:
                            packed_items.append(str(clusters[clave][i]+1))
                            bin_weight += data["weights"][i]
                            bin_value += data["values"][i]
                    print(f"Clientes empacados: {', '.join(packed_items)}")
                    print(f"Volumen empaquetado en el camión: {round(bin_weight, 4)}\n")
                    #print(f"Valor empaquetado del camión: {bin_value}\n")
                    total_value += bin_value
                    total_weight += bin_weight
                    if bin_value > max_value:
                        max_value = bin_value
                        max_value_bin = b

                trucks_used += len(used_bins)
                incomplete = False

            # Print solution for the new bin (if exists)



        else:
            print("No existe solución óptima.")
end_time=time.time()-start_time

print(f"Camiones totales usados: {trucks_used}")
print(f"{end_time} segundos")

**********CLUSTER # 1**********

Número de camiones requeridos: 1

Valor total empaquetado: 15.16

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 7, 8, 15, 24, 37, 45, 51, 62, 86, 89, 105
Volumen empaquetado en el camión: 15.1578

**********CLUSTER # 2**********

Número de camiones requeridos: 1

Valor total empaquetado: 7.26

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 14, 21, 23, 73, 76, 77, 82, 95, 100, 104
Volumen empaquetado en el camión: 7.257

**********CLUSTER # 3**********

Número de camiones requeridos: 1

Valor total empaquetado: 6.38

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 3, 12, 22, 33, 41, 48, 78, 79, 80
Volumen empaquetado en el camión: 6.3791

**********CLUSTER # 4**********

Número de camiones requeridos: 6

Valor total empaquetado: 71.54

*** CAMIÓN # 1 ***
Capacidad: 27
Clientes empacados: 1, 2, 4, 5, 6, 10, 11, 13, 16, 17, 18, 19, 25
Volumen empaquetado en el camión: 14.5185

*** CAMIÓN # 2 ***
Capacidad: 27
Clientes empacados: 26, 27, 2

In [79]:

packed_items = [int(x) for x in packed_items]
mini_tcps = []
for i in clusters:
    print(i)
    i = list(clusters[i]) + [0]

    mini_tcps.append(result_array[i,:][:,i])

mini_tcps[1]

0
1
2
3
4


array([[   0, 1440, 1500, 1260, 1200, 2280,  960, 1320,  840, 1380, 1140],
       [1440,    0,  360,  660, 1440, 1140, 1620, 1320, 1380, 1020, 1500],
       [1680,  540,    0,  780, 1680,  900, 1860, 1560, 1620, 1080, 1560],
       [1200,  600,  480,    0, 1320, 1320, 1380, 1200, 1200,  600, 1200],
       [1200, 1440, 1440, 1260,    0, 2220,  780,  240,  660, 1320,  360],
       [2220, 1020,  900, 1320, 2220,    0, 2400, 2100, 2160, 1620, 2280],
       [1080, 1680, 1680, 1500,  840, 2460,    0,  960,  420, 1560,  900],
       [1440, 1380, 1440, 1200,  240, 2160,  960,    0,  840, 1320,  420],
       [ 900, 1560, 1560, 1380,  720, 2340,  540,  840,    0, 1440,  780],
       [1380,  960,  900,  540, 1320, 1680, 1560, 1200, 1380,    0, 1260],
       [1080, 1560, 1440, 1260,  300, 2340,  720,  420,  660, 1320,    0]],
      dtype=int64)